In [1]:
import torch
import utils
import os
import numpy as np
from collections import OrderedDict
from vgg_face_dag import vgg_face_dag

SKIP_LAYERS = utils.SKIP_LAYERS

device = torch.device('cuda')
vgg_weight = './vgg_face_dag.pth'

# ami model
attribute_model = vgg_face_dag(vgg_weight)
attribute_model.to(device)
attribute_model.eval()
attribute_model.set_ami_params(100., 2.15, 60.)
attribute_model.register_my_hook(skip_layers=SKIP_LAYERS, ami_data=utils.load_neuron_set_lists())

# original model
vgg_net = vgg_face_dag(vgg_weight)
vgg_net.to(device)
vgg_net.eval()

register hook for conv1_1
register hook for conv1_2
register hook for pool1
register hook for conv2_1
register hook for conv2_2
register hook for pool2
register hook for conv3_1
register hook for conv3_2
register hook for conv3_3
register hook for pool3
register hook for conv4_1
register hook for conv4_2
register hook for conv4_3
register hook for pool4
register hook for conv5_1
register hook for conv5_2
register hook for conv5_3
register hook for pool5
register hook for fc6
register hook for fc7


Vgg_face_dag(
  (conv1_1): Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu1_1): ReLU(inplace=True)
  (conv1_2): Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu1_2): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu2_1): ReLU(inplace=True)
  (conv2_2): Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu2_2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU(inplace=True)
  (conv3_2): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_2): ReLU(inplace=True)
  (conv3_3): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (relu3_3): ReLU(inplace=True)
  (pool3)

In [3]:
vgg_root   = '../../data/vgg_face_caffe/'
vgg_names  = utils.read_list(vgg_root + 'names.txt')

In [5]:
img_dir = '/tmp/sized_full_images/'
img_list = utils.read_list('./benign_list.txt')

img_count = 0
adv_count = 0

with torch.no_grad():
    for img_name in img_list:
        img_name  = img_name.strip()
        img_path  = os.path.join(img_dir, img_name)

        prob_original  = vgg_net(utils.get_data(img_path).to(device))
        prob_attribute = attribute_model(utils.get_data(img_path).to(device))
        id_original    = torch.max(prob_original, 1)[1].item()
        id_attribute   = torch.max(prob_attribute, 1)[1].item()

        id_gold = utils.get_identity(img_name, vgg_names)
        
        if id_gold == id_original:
            img_count += 1
        
            ### consistency observer for detecting adversarial samples ###
            if id_original != id_attribute:
                adv_count += 1

            print('{:3} Gold-Original-Attribute: {:4}-{:4}-{:4}  |  Adversary: {:.4f} [{:3}/{:3}]'
                  .format(img_count, id_gold, id_original, id_attribute, 
                          1.0*adv_count/img_count, adv_count, img_count))

  1 Gold-Original-Attribute: 2019-2019-2019  |  Adversary: 0.0000 [  0/  1]
  2 Gold-Original-Attribute: 1992-1992-1992  |  Adversary: 0.0000 [  0/  2]
  3 Gold-Original-Attribute:  451- 451- 451  |  Adversary: 0.0000 [  0/  3]
  4 Gold-Original-Attribute: 2370-2370-2370  |  Adversary: 0.0000 [  0/  4]
  5 Gold-Original-Attribute:  134- 134- 134  |  Adversary: 0.0000 [  0/  5]
  6 Gold-Original-Attribute:  213- 213- 213  |  Adversary: 0.0000 [  0/  6]
  7 Gold-Original-Attribute: 2581-2581-2581  |  Adversary: 0.0000 [  0/  7]
  8 Gold-Original-Attribute:  892- 892- 892  |  Adversary: 0.0000 [  0/  8]
  9 Gold-Original-Attribute: 1754-1754-1754  |  Adversary: 0.0000 [  0/  9]
 10 Gold-Original-Attribute:  586- 586- 586  |  Adversary: 0.0000 [  0/ 10]
 11 Gold-Original-Attribute: 1600-1600-1047  |  Adversary: 0.0909 [  1/ 11]
 12 Gold-Original-Attribute: 2455-2455-2455  |  Adversary: 0.0833 [  1/ 12]
 13 Gold-Original-Attribute: 2051-2051-2051  |  Adversary: 0.0769 [  1/ 13]
 14 Gold-Ori

116 Gold-Original-Attribute: 1129-1129-1129  |  Adversary: 0.0776 [  9/116]
117 Gold-Original-Attribute:  442- 442- 442  |  Adversary: 0.0769 [  9/117]
118 Gold-Original-Attribute:  727- 727- 727  |  Adversary: 0.0763 [  9/118]
119 Gold-Original-Attribute: 1230-1230-1230  |  Adversary: 0.0756 [  9/119]
120 Gold-Original-Attribute: 1797-1797-1797  |  Adversary: 0.0750 [  9/120]
121 Gold-Original-Attribute: 1348-1348-1348  |  Adversary: 0.0744 [  9/121]
122 Gold-Original-Attribute:  818- 818- 818  |  Adversary: 0.0738 [  9/122]
123 Gold-Original-Attribute:  140- 140- 140  |  Adversary: 0.0732 [  9/123]
124 Gold-Original-Attribute: 1648-1648-1648  |  Adversary: 0.0726 [  9/124]
125 Gold-Original-Attribute:  514- 514- 514  |  Adversary: 0.0720 [  9/125]
126 Gold-Original-Attribute: 1416-1416-1416  |  Adversary: 0.0714 [  9/126]
127 Gold-Original-Attribute: 1872-1872-1872  |  Adversary: 0.0709 [  9/127]
128 Gold-Original-Attribute: 2367-2367-2367  |  Adversary: 0.0703 [  9/128]
129 Gold-Ori

229 Gold-Original-Attribute:  177- 177- 177  |  Adversary: 0.0437 [ 10/229]
230 Gold-Original-Attribute: 2112-2112-2112  |  Adversary: 0.0435 [ 10/230]
231 Gold-Original-Attribute:  781- 781- 781  |  Adversary: 0.0433 [ 10/231]
232 Gold-Original-Attribute: 2098-2098-2098  |  Adversary: 0.0431 [ 10/232]
233 Gold-Original-Attribute: 1485-1485-1485  |  Adversary: 0.0429 [ 10/233]
234 Gold-Original-Attribute: 1796-1796-1796  |  Adversary: 0.0427 [ 10/234]
235 Gold-Original-Attribute:  966- 966- 966  |  Adversary: 0.0426 [ 10/235]
236 Gold-Original-Attribute:  656- 656- 656  |  Adversary: 0.0424 [ 10/236]
237 Gold-Original-Attribute:  341- 341- 341  |  Adversary: 0.0422 [ 10/237]
238 Gold-Original-Attribute: 1421-1421-1584  |  Adversary: 0.0462 [ 11/238]
239 Gold-Original-Attribute: 2212-2212-2212  |  Adversary: 0.0460 [ 11/239]
240 Gold-Original-Attribute:  263- 263- 263  |  Adversary: 0.0458 [ 11/240]
241 Gold-Original-Attribute:  701- 701- 701  |  Adversary: 0.0456 [ 11/241]
242 Gold-Ori